In [2]:
!pip install pypdf2
!pip install pdfminer.six
!pip install ftfy

!pip install git+https://github.com/LIAAD/yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-oa5_zc_x
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-oa5_zc_x
  Resolved https://github.com/LIAAD/yake to commit 0fa58cceb465162b6bd0cab7ec967edeb907fbcc
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 6.3 MB/s eta 0:00:00
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=61997 sha256=7876ebded522415b1a41b1be680a7a187af7bf2fcf3e863f4de6af2cf3e35bcb
  Stored in directory: /tmp/pip-ephem-wheel-cache-4sp6u6za/wheels/06/e6/1c/8f614adcd4b053020d672b9cbc5ef1166407755a4b71adea0d
Successfully built yake


In [3]:
import PyPDF2
import re
import string

from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
from tqdm import tqdm_notebook

import ftfy
import yake
import nltk
import json
nltk.download('wordnet')

from google.colab import drive
drive.mount('/content/gdrive')

[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/gdrive


# File processing

In [39]:
lemma = nltk.wordnet.WordNetLemmatizer()
class MCQDataCollector:
  def __init__(self):
    self.collection = []
    self.books_terms = []
    self.term_to_context_storage={}

  def pdf_to_text(self, pdf_file):
    with open(pdf_file, 'rb') as f:
      reader = PyPDF2.PdfReader(f, strict=False)
      pdf_text=[]

      for page in reader.pages:
        content = page.extract_text()
        pdf_text.append(content)
      return pdf_text

  def add_to_collection(self, pdf_file, start_page=0, end_page=None):
    # skan and decompose into pages
    page_char_width, books_contents = self.pdf_to_text_adv(pdf_file, start_page=start_page, end_page=end_page)

    books_contents_main = self.extracting_plain_text(books_contents)[1]
    books_contents_cleaned = [remove_next_line(remove_special(fix_ligatures(p))) for p in books_contents_main]

    self.collection.append(books_contents_cleaned)

  def text_extraction(self, element):
      # extract text from element
      line_text = element.get_text()

      # Getting all formats in text

      line_formats = []
      for text_line in element:
          if isinstance(text_line, LTTextContainer):
              # for each symbol get its format
              for character in text_line:
                  if isinstance(character, LTChar):
                      # font name
                      line_formats.append(character.fontname)
                      # font size
                      line_formats.append(character.size)
      # unique fonts
      format_per_line = list(set(line_formats))

      # tuple of text and its unique formats
      return (line_text, format_per_line)

  def pdf_to_text_adv(self, pdf_file, start_page=0, end_page=None):
    # pdf_file: file we want to scan
    # start_page: number of page-1 to start the scan from
    # end_page: number of page-1 to stop the scan on
    max_char = -1
    n_pages = len(self.pdf_to_text(pdf_file))

    end_page = n_pages if end_page is None else n_pages+end_page if\
      end_page < 0 else end_page

    with open(pdf_file, 'rb') as f:
      reader = PyPDF2.PdfReader(f, strict=False)
      pdf_text=[]
      text_per_page = {}
      try:
        for pagenum, page in tqdm_notebook(enumerate(extract_pages(pdf_file))):
          if pagenum<start_page:
            continue
          if pagenum>end_page:
            continue
          pagecontent = reader.pages[pagenum]

          page_text = []
          line_format = []
          #text_from_images = []
          #text_from_tables = []
          page_content = []
          # Находим все элементы
          page_elements = [(element.y1, element) for element in page._objs]
          # Sort page elements as they present on it
          page_elements.sort(key=lambda a: a[0], reverse=True)

          for i,component in enumerate(page_elements):

            # Extracting page element
            element = component[1]

            # Check if page element is text
            if isinstance(element, LTTextContainer):
                # extracting text rows and their formats
                (line_text, format_per_line) = self.text_extraction(element)
                # text of each row goes to page text list
                page_text.append(line_text)
                max_char = max([max_char, max([len(s) for s in line_text.split('\n')])])
                # adding formats
                line_format.append(format_per_line)
                page_content.append(line_text)


          dctkey = 'Page_'+str(pagenum)
          # list of page elements
          text_per_page[dctkey]= [page_text, line_format, page_content]
      except Exception as e:
        print(e)

    return max_char, text_per_page


  def extracting_plain_text(self, books_contents, min_row_length=50):
    # extracting main text in book -- the one with most populated font
    # text from figures captions which is related to examples is most likely ignored
    # and doesn't count to what will be used for text generation
    dict_font_stats={}

    text_by_fonts={}

    for ipg, page_structure in books_contents.items():
      text_list = page_structure[0]
      fonts_list = page_structure[1]
      for row, row_fonts in zip(text_list, fonts_list):
        if len(row)>=min_row_length:

          all_fonts_in_row = [str(round(v, 3)) for v in row_fonts if not isinstance(v, str)]
          # выбираем именно числовые значения высоты шрифта без привязки к шрифту
          for font_size in set(all_fonts_in_row):
            if font_size not in dict_font_stats.keys():
              dict_font_stats[font_size]=1
              text_by_fonts[font_size]=[row]
            else:
              dict_font_stats[font_size]+=1
              text_by_fonts[font_size]+=[row]
    freq_font_size = max(dict_font_stats, key=dict_font_stats.get)
    return freq_font_size, text_by_fonts[freq_font_size]

  def collect_key_terms(self, scanning_window_width=400, scanning_step=200):
    # window and step are measured in num. of characters

    all_keywords = []

    kw_extractor = yake.KeywordExtractor()
    for doc in self.collection:

      #text = ' '.join(doc)
      i=0

      keywords_by_window = []

      while i < len(doc):
        text = ' '.join(doc[i:i+scanning_window_width])

        keywords = kw_extractor.extract_keywords(text.lower())
        keywords = keywords[:10]# extracting top 10 determined keywords ranged by the method
        keywords_by_window.append(keywords)
        i+=scanning_step

      all_keywords+=list(set([v[0]  for window in keywords_by_window for v in window]))
    keywords_processed = keywords_step2(list(set(all_keywords)))

    self.books_terms+=keywords_processed

    return keywords_processed

  def extract_contexts(self):

    for term in self.books_terms:
      for doc in self.collection:
        for idx, passage in enumerate(doc):
          if term in passage.lower():
            contexts = passage
            if term in self.term_to_context_storage.keys():
              self.term_to_context_storage[term]+=[contexts]
            else:
              self.term_to_context_storage[term]=[contexts]

  def save_extraction_results(self, save_file):
    with open(save_file, 'w') as fp:
        json.dump(self.term_to_context_storage, fp)

def keywords_step2(keywords_list):
  res = []
  skip = []
  normalized_compounds_2 = []
  normalized_compounds_3 = []
  for kw in keywords_list:
    if kw not in skip:
      words_in_kw = kw.split(' ')
      if len(words_in_kw)>1:
        w1 = ' '.join([w for w in words_in_kw[:-1]])+' '+lemma.lemmatize(words_in_kw[-1])
        if len(words_in_kw) == 2:
          normalized_compounds_2.append(w1)
          res.append(w1)
        else:
          normalized_compounds_3.append(w1)
      else:
        normalized = lemma.lemmatize(kw)
        res.append(normalized)

  for w in normalized_compounds_3:
    if len([x for x in normalized_compounds_2 if x in w]) == 0:
      res.append(w)
  return res

# some text processing functions
def fix_ligatures(text):
  return ftfy.fixes.fix_latin_ligatures(text)

def remove_special(page):
  page = re.sub(r'’', '`', page)
  filtered_string = filter(lambda x: x in string.printable, page)
  return ''.join([x for x in filtered_string])

def remove_next_line(page):
  res = re.sub(r'-\n', '', page)
  return re.sub(r'[\r\n]+', ' ', res) #''.join([x if x!='\n' else ' ' for x in res])

def specify_content_location(contents, start, end=None):
  return contents[start:] if end is None else contents[start:end]

## Extracting data from a book

In [40]:
bookreader = MCQDataCollector()
bookreader.add_to_collection('/content/gdrive/MyDrive/Colab Notebooks/NLP_LLMagent/ed3book.pdf', start_page=8, end_page=650)

<ipython-input-39-b5b3539b1f86>:64: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for pagenum, page in tqdm_notebook(enumerate(extract_pages(pdf_file))):


0it [00:00, ?it/s]

In [41]:
res1 = bookreader.collect_key_terms()

In [42]:
bookreader.extract_contexts()

In [ ]:
##

# LLM instantiation for contexts selection

In [1]:
!pip install datasets

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm
import json
import random
import ast

import numpy as np
import pandas as pd

In [4]:
with open('/content/drive/MyDrive/Colab Notebooks/NLP_LLMagent/book_contents_v2.json', 'r') as fp:
    contents = json.load(fp)

In [5]:
torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct",
    device_map="cuda",
    #device_map='cpu',
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", padding_side='left')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
terms = list(contents.keys())

In [ ]:
batch_size = 8
pipe = pipeline(
    "text-generation",
    #"question-answering",
    model=model,
    tokenizer=tokenizer,
    batch_size=batch_size
)

generation_args = {
    "max_new_tokens": 250,
    "return_full_text": False,
    "temperature": 0.2,
    "do_sample": True,
}


## Context selector code for cases with large number of contexts for term
Format: tournament, contexts are compared given term. Then reduced set goes for further selection based on relevance.

In [ ]:
message_compare = lambda q: [
    {"role": "system", "content": """You are a helpful assistant.
    Given a term and two passages answer which passage is most relevent to the term. Answer with passage number."""},
    {"role": "user", "content": q}
]

text = lambda msg: tokenizer.apply_chat_template(
    msg,
    tokenize=False,
    add_generation_prompt=True
)

def generate_pairs(a):
    if len(a) % 2 == 0:
        for i in range(0, len(a), 2):
            yield a[i:(i+2)]
    else:
        if len(a) == 3:
            yield a
        else:
            a1, a2 = a[:-3], a[-3:]
            yield from generate_pairs(a1)
            yield a2
random.seed(42)
def matchmaker(alternatives, random_state=42):
    random.seed(random_state)
    random.shuffle(alternatives)
    return alternatives

def pairwise_topk(alternatives, term, k=4):
    if len(alternatives) < 2*k:
        return alternatives
    res = matchmaker(alternatives)
    while len(res) > 2*k:
        res_new = []
        for pair in generate_pairs(res):
            if len(pair)==2:
                #print(llm_pair_choice(pair))
                best_of_pair, _ = llm_pair_choice(pair, term)
            else:
                best_of_pair = best_of_3(pair, term)
            res_new.append(best_of_pair)
        res = res_new+[]

    return res

def best_of_3(alternatives, term):
    a1, a2, a3 = tuple(alternatives)
    winner1, loser1 = llm_pair_choice([a1, a2], term)
    winner2, loser2 = llm_pair_choice([a3, loser1], term)
    if winner2 == loser1:
        return winner1
    else:
        winner = llm_pair_choice([winner1, winner2], term)[0]
        return winner

#message
def llm_pair_choice(ps, term):
    response = pipe(text(message_compare(f"Term: {term}, Passage1: {ps[0]}, Passage2: {ps[1]}")), **generation_args)[0]['generated_text']
    return (ps[0], ps[1]) if '1' in response else (ps[1], ps[0])

## Few-shot prompt for context filtering based on relevance
Relevance of a context for a term is determined by context describing general functions, applications. If the term is used by some other object, it's also considered. Examples in prompt describe these ideas.

In [ ]:
#few-shot relevance selector
message_validate_explain = lambda q: [
    {"role": "system", "content": """
    Using only the information in the context and given term tell if the context describes
     general functions or applications including those in broader field of the term.
     Respond with 'Yes' or 'No' and explain the answer.
     """},
    {"role": "user", "content": """
        Term: engine.
    Context: In order to make vehicle moving faster one has to develop such engine that will be powerful enough.
    Response: Yes. Explanation: The context describes the term 'engine' as an functioning part of vehicles which makes latter moving.

    Term: code.
    Context: In this example above we see that the code throws errors.
    Response: No. Explanation: The context doesn't describe neither functions not applications of code since it refers to some example which is not included in the context.

    Term: parallel computation.
    Context: For example, you can apply parallel computation approach to calculate the sum of large sequence.
    Response: Yes. Explanation: The context describes an example of an application of parallel computations.

    """+q}
]

In [ ]:
from tqdm import tqdm_notebook
from IPython.display import clear_output
k=256
def select_best_chunks(terms, contents, k_terms_limit=40):
    selected = {}
    for i, term in enumerate(terms[:k_terms_limit]):
        print(f'Status: {i+1}/{len(terms)} terms processed')
        selected[term]=[]


        if len(contents[term]) > k:
            # apply the matchmaker-like choice
            contexts = pairwise_topk(contents[term], term, k=k//2)
        else:
            contexts = contents[term]

        dataset = Dataset.from_dict({
            'index': list(range(0, len(contexts))),

            'Context': [text(message_validate_explain(f"Term: '{term}', Context: {cont}. Response: ")) for j, cont in enumerate(contexts)]
        })
        j=0
        for out in tqdm(pipe(KeyDataset(dataset, 'Context'), **generation_args),
                total=len(contexts)):
            response = out[0]['generated_text']
            if 'Yes' in response:
                selected[term].append(contexts[j])
            j+=1

        clear_output(3)
    return selected
sorted_content = dict(sorted(contents.items(), key=lambda x: len(x[1])))
sorted_terms = list(sorted_content.keys())

selected_chunks = select_best_chunks(sorted_terms, contents)

In [ ]:
with open('book_contents_selected.json', 'w') as fp:
    json.dump(selected_chunks, fp)

# Questions generation

In [6]:
with open('/content/drive/MyDrive/Colab Notebooks/NLP_LLMagent/book_contents_selected.json', 'r') as fp:
    selected_chunks = json.load(fp)

## Overall code for agent class
Includes:
- prompts for first 3 Bloom question types
- class which uses LLM to generate **multiple-choice questions (MCQ)** of each type given context and term

Prompts were chosen after comparison between zero-shot and one/few-shot by LLM Judge (separate notebook)

Bloom's taxonomy indicators for each type are taken from [https://www.swtxc.edu/documents/oipr/bloom.pdf]

In [7]:
files_paths = {'term_contexts': '/content/drive/MyDrive/Colab Notebooks/NLP_LLMagent/book_contents_v2.json',
                'selected_term_contexts': '/content/drive/MyDrive/Colab Notebooks/NLP_LLMagent/book_contents_selected.json',
                'gen_mcq_storage': None}

BLOOM_REMEMBER_PROMPT=["""
    Using the information in the context and given term generate multiple-choice question
    related to the term according to context that doesn't have answer in question's text and includes question words like "What","Why" or "How"
    in format
    {
        'Question': "...",
        'Options':
        {
        'A)': "...",
        'B)': "...",
        'C)': "...",
        'D)': "..."
        },
        'Correct answer': ...
    }
    Question's text must be without correct answer and without words like 'they', 'according to he context', 'context provided' or 'given context'.
""", '']

BLOOM_COMPREHENSION_PROMPT =["""
    Using the information in the context and given term generate
    multiple-choice question related to the term according to the context that tests reader's comprehension and
    includes a verb like 'explain', 'summarize', 'rephrase' or 'compare'
    in form
    {
        'Question': "...",
        'Options':
        {
        'A)': "...",
        'B)': "...",
        'C)': "...",
        'D)': "..."
        },
        'Correct answer': ...
    }
    Question's text must be without correct answer and without words like 'they', 'according to he context', 'context provided' or 'given context'.
""",
"""
    Example 1:
    Term: termination process.
    Context: It is reported that evaluating achieved goals and accomplishements describes the termination process.

    {
        'Question': "Summarize what the termination process is MOST often associated with?",
        'Options':
        {
        'A)': "Evaluating goals and accomplishments",
        'B)': "Implementing a plan of action",
        'C)': "Setting limits",
        'D)': "Developing a contract"
        },
        'Correct answer': 'A)'
    }

    Example 2:
    Term: {term}
    Context: While {object} is used in such tasks as {task1} and {task2} the main application is related to {task3}.

    {
        'Question': "Compare {task1} and {task3} in terms of application of {object} to solve them?",
        'Options':
        {
        'A)': "{object} is used primarily in {task3} as opposed to {task1}",
        'B)': "{object} is used primarily in {task1} as opposed to {task3}",
        'C)': "{object} is not mainly focused on solving both tasks",
        'D)': "None/All of the above"
        },
        'Correct answer': 'A)'
    }

    Your turn:

    """]

BLOOM_APPLICATION_PROMPT = ["""
    Using the information in the context and given term generate
    multiple-choice question related to the term according to the context that tests reader's ability to apply concept and
    includes a verb like 'develop', 'organize', 'build' or 'plan'
    in form
    {
        'Question': "...",
        'Options':
        {
        'A)': "...",
        'B)': "...",
        'C)': "...",
        'D)': "..."
        },
        'Correct answer': ...
    }
    Question's text must be without correct answer and without pronouns or words like 'according to the context', 'context provided' or 'given context'.
    """,
    """
    Example 1:
    Term: {term}
    Context: In order to do get {object1} from {object2} one has to apply {method1} and then process its result by {method2}.

    {
        'Question': "How would you organize the pipeline where you have to transform {object1} into {object2}",
        'Options':
        {
        'A)': "Just apply {method1} to {object1}",
        'B)': "Just apply {method2} to {object1}",
        'C)': "Apply {method2} and then use its result in {method1}",
        'D)': "Apply {method1} and then use its result in {method2}"
        },
        'Correct answer': 'D)'
    }

    Your turn:

    """]

type_to_prompt_map ={
    'remember': BLOOM_REMEMBER_PROMPT,
    'comprehension': BLOOM_COMPREHENSION_PROMPT,
    'application': BLOOM_APPLICATION_PROMPT
}

type_to_complexity_map ={
    'remember': 0,
    'comprehension': 1,
    'application': 2
}

class MCQ_generator():
    def __init__(self, base_llm, tokenizer, files_paths=None, batch_size=9):
        self.base_llm = base_llm
        self.tokenizer = tokenizer
        self.files_paths = files_paths if files_paths is not None else {'term_contexts': None,
                                                                        'selected_term_contexts': None,
                                                                        'gen_mcq_storage': None}
        self.results_saved = {'term_contexts': None,
                            'selected_term_contexts': None,
                            'gen_mcq_storage': None}
        self.batch_size = batch_size

        self.pipe = pipeline(
            "text-generation",

            model=self.base_llm,
            tokenizer=self.tokenizer,
            batch_size=self.batch_size
        )

        self.generation_args_context_extract = {
            "max_new_tokens": 250,
            "return_full_text": False,
            "temperature": 0.01,
            "do_sample": True,
        }

        self.generation_args = {
            "max_new_tokens": 250,
            "return_full_text": False,
            "temperature": 0.2,
            "do_sample": True,
        }

        self.text = lambda msg: self.tokenizer.apply_chat_template(
            msg,
            tokenize=False,
            add_generation_prompt=True
        )
        self.available_bloom_types = [typ for typ in type_to_complexity_map.keys()]
        self.default_test_template = {k: cnt for k, cnt in zip(self.available_bloom_types, [4,3,3])}

        self.term_stats = self.collect_terms_context_stats()

    def read_results(self):
        if self.files_paths is not None:
            for name, path in self.files_paths.items():
                if path is not None:
                    if '.json' in path:
                        with open(path, 'r') as fp:
                            self.results_saved[name] = json.load(fp)

    def collect_terms_context_stats(self):
        if self.results_saved['selected_term_contexts'] is None:
            return -1
        return {k: len(v) for k, v in self.results_saved['selected_term_contexts'].items()}

    def save_results(self, res_name, file_name, path=None):
        if '.json' in file_name:
            with open(file_name, 'w') as fp:
                json.dump(self.results_saved[res_name], fp)


    def generate_questions(self, question_type='remember'):
        if self.results_saved['selected_term_contexts'] is None:
            return -1

        if question_type not in self.available_bloom_types:
            raise Exception(f"Question type doesn`t belong to available types: {', '.join(self.available_bloom_types)}")
        gen_prompt = type_to_prompt_map[question_type]
        df_data = pd.DataFrame([[term, q] for term, qlist in self.results_saved['selected_term_contexts'].items() for q in qlist],
            columns=['Term', 'Context'])

        prompt_func = lambda q: [
                {"role": "system", "content": gen_prompt[0]},
                {"role": "user", "content": q if len(gen_prompt) < 2 else gen_prompt[1]+'\n'+q}
            ]

        dataset = Dataset.from_dict({
            'index': list(range(0, df_data.shape[0])),

            'Message': [self.text(prompt_func(f"""Term: '{row["Term"]}', Context: {row["Context"]}. Response: """)) for j, row in df_data.iterrows()]
        })

        result = {
            'ALL_TERMS': [],
            'questions_data':{

            },
            'require_validation':[]
        }

        if self.results_saved['gen_mcq_storage'] is None:
            self.results_saved['gen_mcq_storage'] = {
                                        'ALL_TERMS': [],
                                        'questions_data':{

                                        },
                                        'require_validation':[]
                                    }

        generation_info = {'successful_generation_cnt': 0, 'failed_generation_cnt': 0}

        for j, out in tqdm(enumerate(self.pipe(KeyDataset(dataset, 'Message'), **self.generation_args)),
            total=df_data.shape[0]):
            row = df_data.loc[j, :]
            term, context = row["Term"], row["Context"]


            response = out[0]['generated_text'] # expected dict with question, options and answer
            try:
                if response.count('{') != response.count('}'): #needs manual check
                    raise Exception("Unmatched brackets")
                start_i = response.find('{')
                end_i = response.rfind('}')
                question_dict = ast.literal_eval(response[start_i:end_i+1])

                question = question_dict['Question']
                correct_answer_letter = question_dict['Correct answer'][:2]
                options = [question_dict['Options'][correct_answer_letter]]\
                            +[option for i_opt, option in question_dict['Options'].items() if i_opt != correct_answer_letter]

                if term not in self.results_saved['gen_mcq_storage']['ALL_TERMS']:
                    self.results_saved['gen_mcq_storage']['ALL_TERMS'].append(term)
                    self.results_saved['gen_mcq_storage']['questions_data'][term]={
                        'max_id': -1,
                        'Questions':[]
                    }
                self.results_saved['gen_mcq_storage']['questions_data'][term]['max_id']+=1
                curr_id = self.results_saved['gen_mcq_storage']['questions_data'][term]['max_id']
                new_question_dict = {
                    'id': curr_id,
                    'context': context,
                    'Bloom_type': question_type,
                    'question': question,
                    'options': options,
                    'complexity': type_to_complexity_map[question_type]
                }

                self.results_saved['gen_mcq_storage']['questions_data'][term]['Questions'].append(new_question_dict)
                generation_info['successful_generation_cnt'] +=1

            except Exception:
                self.results_saved['gen_mcq_storage']['require_validation'] += [{
                        'term': term,
                        'context': context,
                        'Bloom_type': question_type,
                        'question_body': response
                    }]
                generation_info['failed_generation_cnt'] +=1

        return self.results_saved['gen_mcq_storage'], generation_info

    def generate_test(self, n_questions=10, random_state=42, test_type_template=None):
        if self.results_saved['gen_mcq_storage'] is None:
            return -1
        if test_type_template is None:
           test_type_template = self.default_test_template

        np.random.seed(random_state)
        questions = self.results_saved['gen_mcq_storage']

        terms_choice = np.random.choice(questions['ALL_TERMS'], size=n_questions)

        test_template_list = sum([v*[k] for k, v in test_type_template.items()], start=[])
        terms_diff_pairs = [(t, d) for t, d in zip(terms_choice, test_template_list)]

        terms_q_cnt={str(k): v for k, v in Counter(terms_diff_pairs).items()}

        selected = {}
        j=0
        for (term, diff), freq in terms_q_cnt.items():
          term_q_list = [q for q in questions['questions_data'][term]["Questions"] if q['complexity']==diff]

          chosen = np.random.choice(term_q_list, size=freq, replace=False)
          #print(term, freq)
          for q in chosen:
            selected[j] = q

            options = chosen[0]['options']
            letters = list('ABCD')
            permutation = np.random.permutation(4)
            corr_letter = np.argwhere(permutation == 0)[0][0]
            options_shuffled = [options[k] for k in permutation]
            selected[j]['shuffled_options'] = options_shuffled
            selected[j]['answer'] = letters[corr_letter]

            j+=1
        return selected

        #mode: questions are still random but not mixed (correct answer is first among options)

In [8]:
mcq_gen = MCQ_generator(model, tokenizer, files_paths, batch_size=16)
mcq_gen.read_results()

Device set to use cuda


In [9]:
remember_res, remember_gen_info = mcq_gen.generate_questions(question_type='remember')
remember_comp_res, comp_gen_info = mcq_gen.generate_questions(question_type='comprehension')
all_res, app_gen_info = mcq_gen.generate_questions(question_type='application')

100%|██████████| 681/681 [25:15<00:00,  2.22s/it]


In [10]:
mcq_gen.save_results('gen_mcq_storage', 'gen_mcq_storage_v2.json')

# Launching test interface

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [2]:
import gradio as gr
import json
import numpy as np
from collections import Counter
import time

In [3]:
with open('/content/drive/MyDrive/Colab Notebooks/NLP_LLMagent/gen_mcq_storage_v2.json', 'r') as fp:
  questions = json.load(fp)

In [4]:
test_template = {0:4, 1: 3, 2: 3}

In [5]:
np.random.seed(42)
n_questions=10

terms_choice = np.random.choice(questions['ALL_TERMS'], size=n_questions)
test_template_list = sum([v*[k] for k, v in test_template.items()], start=[])
terms_diff_pairs = [(t, d) for t, d in zip(terms_choice, test_template_list)]

terms_q_cnt={str(k): v for k, v in Counter(terms_diff_pairs).items()}
terms_q_cnt

{"(np.str_('translation'), 0)": 1,
 "(np.str_('naive bayes'), 0)": 1,
 "(np.str_('temporal expression'), 0)": 1,
 "(np.str_('mention detection'), 0)": 1,
 "(np.str_('hidden layer'), 1)": 1,
 "(np.str_('translation'), 1)": 1,
 "(np.str_('semantic role'), 1)": 1,
 "(np.str_('speech recognition'), 2)": 1,
 "(np.str_('word meaning'), 2)": 2}

In [6]:
np.random.seed(42)

terms_choice = np.random.choice(questions['ALL_TERMS'], size=n_questions)
test_template_list = sum([v*[k] for k, v in test_template.items()], start=[])
terms_diff_pairs = [(t, d) for t, d in zip(terms_choice, test_template_list)]

terms_q_cnt={k: v for k, v in Counter(terms_diff_pairs).items()}
selected = {}
j=0
for (term, diff), freq in terms_q_cnt.items():
  term_q_list = [q for q in questions['questions_data'][term]["Questions"] if q['complexity']==diff]

  chosen = np.random.choice(term_q_list, size=freq, replace=False)
  #print(term, freq)
  for q in chosen:
    selected[j] = q

    options = chosen[0]['options']
    letters = list('ABCD')
    permutation = np.random.permutation(4)
    corr_letter = np.argwhere(permutation == 0)[0][0]
    options_shuffled = [options[k] for k in permutation]
    selected[j]['shuffled_options'] = options_shuffled
    selected[j]['answer'] = letters[corr_letter]

    j+=1

In [7]:
def check_answer(choice):
  global i, selected, n_correct

  if choice == selected[i]['answer']:
    n_correct+=1
    updated_verdict = gr.Textbox(lines=2, visible=True, value='Correct answer')
    update_nextq = gr.Button(value='Next question', interactive=True)
    updated_submit = gr.Button(value='Submit', interactive=False)
    updated_hint = gr.Button(value='Show hint?', interactive=False)

    return updated_verdict, update_nextq, updated_submit, updated_hint
  else:
    updated_verdict = gr.Textbox(lines=2, visible=True, value='Incorrect answer')
    update_nextq = gr.Button(value='Next question', interactive=True)
    updated_submit = gr.Button(value='Submit', interactive=False)
    updated_hint = gr.Button(value='Show hint?', interactive=True)

    return updated_verdict, update_nextq, updated_submit, updated_hint

def next_question_switch():
  global i, n_questions, selected
  i+=1

  if i < n_questions:
    title_text=f"MCQ test, Progress {i+1}/{n_questions}"
    update_nextq = gr.Button(value='Next question', interactive=False)
    updated_submit = gr.Button(value='Submit', interactive=False)
    updated_hint = gr.Button(value='Show hint?', interactive=False)

    cleared_hint_box = gr.Textbox(value='', label='Hint')

    updated_options = []

    for letter, option in zip('ABCD', selected[i]['shuffled_options']):
        updated_options.append(gr.Textbox(label=letter, value=option))

    updated_question = gr.Radio(
              [s for s in 'ABCD'], label=selected[i]['question'],
              min_width=160, value=None
          )
    info_box = gr.Textbox(label='Info', value='Choose correct answer')
  else:
    i=-1
    title_text=f"MCQ test, Results"
    update_nextq = gr.Button(value='Retry', interactive=True)
    updated_submit = gr.Button(value='Submit', interactive=False)
    updated_hint = gr.Button(value='Show hint?', interactive=False)
    cleared_hint_box = gr.Textbox(value="", label='Hint')

    updated_options = []

    for letter in 'ABCD':
        updated_options.append(gr.Textbox(label=letter, value=""))

    updated_question = gr.Radio(
              [s for s in 'ABCD'], label="",
              min_width=160, interactive=False
          )
    info_box = gr.Textbox(label='Results', value=f'{n_correct}/{n_questions} correct')
  return info_box, updated_submit, update_nextq, updated_hint, gr.HTML("<h1>"+title_text+"</h1>"), cleared_hint_box,\
            updated_question, *updated_options

def show_hint_action():
  global i, selected
  return gr.Textbox(label=f"Correct answer: {selected[i]['answer']}", value=selected[i]['context'])

def unlock_submit(radio):
  if radio is not None:
    updated_submit = gr.Button(value='Submit', interactive=True)
    return updated_submit
  return gr.Button(value='Submit', interactive=False)

i=0
n_correct = 0
with gr.Blocks() as demo:
  with gr.Row():
    title_text=f"MCQ test, Progress {i+1}/{n_questions}"
    title = gr.HTML("<h1>"+title_text+"</h1>")
    #slider2 = gr.Textbox(label="s2")
    #drop3 = gr.Dropdown(["a", "b", "c"], label="d3")

  options = selected[i]['shuffled_options']
  with gr.Row():
    with gr.Column(min_width=30):
      textboxes = []
      for letter, option in zip('ABCD', options):
        textboxes.append(gr.Textbox(label=letter, value=option))
    with gr.Column(min_width=30):
      choice = gr.Radio(
            [s for s in 'ABCD'], label=selected[i]['question'],
            min_width=160
        )

      with gr.Row():
        with gr.Column(min_width=30):
          hint_and_answer = gr.Textbox(label='Info', value='Choose correct answer')
        with gr.Column(min_width=30):
          submit_button = gr.Button(value='Submit', interactive=False)
          next_question = gr.Button(value='Next question', interactive=False)
          show_hint_button = gr.Button(value='Show hint?', interactive=False)


      hint_box = gr.Textbox(value='', label='Hint')

      submit_button.click(fn=check_answer, inputs=choice, outputs=[hint_and_answer, next_question, submit_button, show_hint_button])

      next_question.click(fn=next_question_switch, outputs=[hint_and_answer,
                                                            submit_button,
                                                            next_question,
                                                            show_hint_button,
                                                            title,
                                                            hint_box,
                                                            choice,
                                                            *textboxes])

      show_hint_button.click(fn=show_hint_action, outputs=[hint_box])

      choice.change(unlock_submit, choice, submit_button)


demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://35c1f7a6d623bec3d5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://35c1f7a6d623bec3d5.gradio.live
